In [42]:
import sys
import numpy as np
import pandas as pd
from pytz import UTC

from logbook import (NestedSetup, NullHandler, Logger, StreamHandler, StderrHandler, 
                     INFO, WARNING, DEBUG, ERROR)

from zipline import run_algorithm
from zipline.api import (attach_pipeline, 
                         date_rules, 
                         time_rules,
                         get_datetime,
                         order_target_percent,
                         pipeline_output, 
                         record, 
                         schedule_function, 
                         get_open_orders, 
                         calendars,
                         set_commission, 
                         set_slippage)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume

from pyfolio.utils import extract_rets_pos_txn_from_zipline

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [43]:
# setup stdout logging
format_string = '[{record.time: %H:%M:%S.%f}]: {record.level_name}: {record.message}'
zipline_logging = NestedSetup([NullHandler(level=DEBUG),
                               StreamHandler(sys.stdout, format_string=format_string, level=INFO),
                               StreamHandler(sys.stderr, level=ERROR)])
zipline_logging.push_application()
log = Logger('Algorithm')

In [44]:
# Settings
MONTH = 21
YEAR = 12 * MONTH
N_LONGS = 1
N_SHORTS = 0
VOL_SCREEN = 500
start = pd.Timestamp('2015-01-01')
end = pd.Timestamp('2017-01-01' )
capital_base = 1e7

In [45]:
class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())

In [46]:
class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())

In [47]:
def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs'  : mean_reversion.bottom(N_LONGS),
                             'shorts' : mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))

In [48]:
def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')
    record(factor_data=context.factor_data.ranking)
    assets = context.factor_data.index
    record(prices=data.current(assets, 'price'))

In [50]:
def rebalance(context, data):
    """Compute long, short and obsolete holdings; place trade orders"""
    factor_data = context.factor_data
    assets = factor_data.index
    
    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = context.portfolio.positions.keys() - longs.union(shorts)
    log.info('{} | Longs: {:2.0f} | Shorts: {:2.0f} | {:,.2f}'.format(get_datetime().date(),
                                                                     len(longs), 
                                                                     len(shorts),
                                                                     context.portfolio.portfolio_value))

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS if N_LONGS else 0)
    exec_trades(data, assets=shorts, target_percent=-1 / N_SHORTS if N_SHORTS else 0)

In [51]:
def exec_trades(data, assets, target_percent):
    """Place orders for assets using target portfolio percentage"""
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)

In [52]:
def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance,
                      date_rules.month_start(),
                      time_rules.market_open(),
                      calendar=calendars.US_EQUITIES)

    set_commission(us_equities=commission.PerShare(cost=0.00075, 
                                                   min_trade_cost=.01))
    set_slippage(us_equities=slippage.VolumeShareSlippage(volume_limit=0.0025, 
                                                          price_impact=0.01))

In [53]:
backtest = run_algorithm(start=start,
                         end=end,
                         initialize=initialize,
                         before_trading_start=before_trading_start,
                         bundle='qa_datalake',
                         capital_base=capital_base)

[ 19:08:09.249404]: INFO: 2015-01-02 | Longs:  1 | Shorts:  0 | 10,000,000.00
[ 19:08:09.352016]: INFO: 2015-02-02 | Longs:  1 | Shorts:  0 | 8,984,673.44
[ 19:08:09.378142]: INFO: 2015-03-02 | Longs:  1 | Shorts:  0 | 9,549,349.12
[ 19:08:09.408686]: INFO: 2015-04-01 | Longs:  1 | Shorts:  0 | 8,974,148.19
[ 19:08:09.437794]: INFO: 2015-05-01 | Longs:  1 | Shorts:  0 | 11,258,101.72
[ 19:08:09.465178]: INFO: 2015-06-01 | Longs:  1 | Shorts:  0 | 11,383,592.98
[ 19:08:09.495813]: INFO: 2015-07-01 | Longs:  1 | Shorts:  0 | 11,073,778.92
[ 19:08:09.586493]: INFO: 2015-08-03 | Longs:  1 | Shorts:  0 | 11,209,629.69
[ 19:08:09.616704]: INFO: 2015-09-01 | Longs:  1 | Shorts:  0 | 10,714,648.67
[ 19:08:09.651171]: INFO: 2015-10-01 | Longs:  1 | Shorts:  0 | 12,371,700.44
[ 19:08:09.682598]: INFO: 2015-11-02 | Longs:  1 | Shorts:  0 | 14,914,834.18
[ 19:08:09.710571]: INFO: 2015-12-01 | Longs:  1 | Shorts:  0 | 16,005,412.80
[ 19:08:09.844151]: INFO: 2016-01-04 | Longs:  1 | Shorts:  0 | 14,